Imports:

In [ ]:
from datasets import load_dataset
from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from pathlib import Path
from llama_index.core import PromptTemplate

Load dataset (saves to local cache):

In [ ]:

ds = load_dataset("JDRJ/kjv-bible")
print(f"Loaded {len(ds['train'])} verses")

Book name mapping (to make references readable)

In [ ]:
book_map = {
    "Ge": "Genesis",
    "Ex": "Exodus",
    "Le": "Leviticus",
    "Nu": "Numbers",
    "De": "Deuteronomy",
    "Jos": "Joshua",
    "Jg": "Judges",
    "Ru": "Ruth",
    "1Sm": "1 Samuel",
    "2Sm": "2 Samuel",
    "1Ki": "1 Kings",
    "2Ki": "2 Kings",
    "1Chr": "1 Chronicles",
    "2Chr": "2 Chronicles",
    "Ezr": "Ezra",
    "Neh": "Nehemiah",
    "Es": "Esther",
    "Jb": "Job",
    "Ps": "Psalms",
    "Pr": "Proverbs",
    "Ec": "Ecclesiastes",
    "So": "Song of Solomon",
    "Is": "Isaiah",
    "Je": "Jeremiah",
    "La": "Lamentations",
    "Eze": "Ezekiel",
    "Da": "Daniel",
    "Ho": "Hosea",
    "Jl": "Joel",
    "Am": "Amos",
    "Ob": "Obadiah",
    "Jon": "Jonah",
    "Mi": "Micah",
    "Na": "Nahum",
    "Hab": "Habakkuk",
    "Zep": "Zephaniah",
    "Hg": "Haggai",
    "Zec": "Zechariah",
    "Mal": "Malachi",
    "Mt": "Matthew",
    "Mk": "Mark",
    "Lk": "Luke",
    "Jn": "John",
    "Ac": "Acts",
    "Ro": "Romans",
    "1Co": "1 Corinthians",
    "2Co": "2 Corinthians",
    "Ga": "Galatians",
    "Eph": "Ephesians",
    "Php": "Philippians",
    "Col": "Colossians",
    "1Th": "1 Thessalonians",
    "2Th": "2 Thessalonians",
    "1Ti": "1 Timothy",
    "2Ti": "2 Timothy",
    "Ti": "Titus",
    "Phm": "Philemon",
    "He": "Hebrews",
    "Ja": "James",
    "1Pe": "1 Peter",
    "2Pe": "2 Peter",
    "1Jn": "1 John",
    "2Jn": "2 John",
    "3Jn": "3 John",
    "Jd": "Jude",
    "Re": "Revelation",
}


Create LlamaIndex Documents with metadata:

In [ ]:
documents = []
for ex in ds["train"]:
    abbr = ex["Book"]
    book_name = book_map.get(abbr, abbr)  # fallback to abbreviation
    ref = f"{book_name} {ex['Chapter']}:{ex['Verse']}"

    doc = Document(
        text=ex["Text"],
        metadata={
            "reference": ref,
            "book": book_name,
            "chapter": ex["Chapter"],
            "verse": ex["Verse"],
            "source": "KJV",
        },
    )
    documents.append(doc)

print(f"Created {len(documents)} documents")


Configure embedding model & LLM (Ollama + llama3.2):

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.llm = Ollama(
    model="llama3.2",  # or "llama3.2:3b" if you pulled a specific tag
    request_timeout=180.0,  # generous timeout for laptop
    temperature=0.1,  # low for factual answers
)

Local persistent Chroma vector store:

In [ ]:
persist_dir = Path("./kjv_vector_index")
chroma_client = chromadb.PersistentClient(path=str(persist_dir / "chroma_db"))
chroma_collection = chroma_client.get_or_create_collection("kjv_bible")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Build (or load) the index – this is the expensive part first time:

In [ ]:
if (persist_dir / "chroma_db").exists() and len(chroma_collection.get()["ids"]) > 0:
    print("Loading existing index...")
    index = VectorStoreIndex.from_vector_store(
        vector_store, storage_context=storage_context
    )
else:
    print("Building new index (this will take 5–20 minutes)...")
    index = VectorStoreIndex.from_documents(
        documents, storage_context=storage_context, show_progress=True
    )
    print("Index built and saved!")

Create query engine & test it:

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=5
)  # retrieve 5 most relevant verses

# Test queries
queries = [
    "What does John 3:16 say?",
    "Summarize the creation story in Genesis 1",
    "What did Jesus say about loving your neighbor?",
    "How many times is 'love' mentioned in 1 Corinthians?",
]

for q in queries:
    print(f"\nQuestion: {q}")
    response = query_engine.query(q)
    print("Answer:", response.response)
    print("Sources:", [node.metadata["reference"] for node in response.source_nodes])

In [ ]:
# Cell: Ask the same question to BOTH versions

question = "What does John 3:16 say?"

print("┌──────────────────────────────────────────────────────────────┐")
print("│                    BASE MODEL (no RAG)                       │")
print("└──────────────────────────────────────────────────────────────┘")

# ------------------- BASE (pure Ollama / no context) -------------------
base_response = Settings.llm.complete(question)  # or .chat() if you prefer chat format
print(base_response.text.strip())
print()

print("┌──────────────────────────────────────────────────────────────┐")
print("│               RAG VERSION (with retrieved verses)            │")
print("└──────────────────────────────────────────────────────────────┘")

rag_response = query_engine.query(question)
print(rag_response.response)
print("\nSources:")
for node in rag_response.source_nodes:
    print(f"  • {node.metadata.get('reference', 'unknown')}")


In [ ]:
question = "Who was Methuselah?"
base_response = Settings.llm.complete(question)  # or .chat() if you prefer chat format
print(base_response.text.strip())
print("--------------------------------------------------------------")

rag_response = query_engine.query(question)
print(rag_response.response)
print("\nSources:")
for node in rag_response.source_nodes:
    print(f"  • {node.metadata.get('reference', 'unknown')}")

In [ ]:
"What does John 3:16 say?"
"Who was Methuselah?"
"Explain the parable of the sower"
"What must I do to be saved?"
"How old was Jesus when he was crucified?"


Our RAG model has been restrained. Let's get some additional info back:

In [ ]:
question = "Who was Methuselah?"

query_engine = index.as_query_engine(
    similarity_top_k=5,
    response_mode="tree_summarize",  # or "refine" — more verbose/explanatory
)

rag_response = query_engine.query(question)
print(rag_response.response)
print("\nSources:")
for node in rag_response.source_nodes:
    print(f"  • {node.metadata.get('reference', 'unknown')}")

Custom prompt:

In [ ]:
from llama_index.core import PromptTemplate

question = "Who was Methuselah?"

qa_prompt_tmpl = PromptTemplate(
    "Context from KJV Bible:\n{context_str}\n\n"
    "Question: {query_str}\n\n"
    "Answer using only the provided KJV context. "
    "Quote the relevant verse(s) exactly, then briefly explain or provide context if it helps understanding — but do not add information from outside the KJV text."
)

query_engine = index.as_query_engine(text_qa_template=qa_prompt_tmpl)

rag_response = query_engine.query(question)
print(rag_response.response)
print("\nSources:")
for node in rag_response.source_nodes:
    print(f"  • {node.metadata.get('reference', 'unknown')}")


In [ ]:
qa_prompt_tmpl = PromptTemplate(
    "You are an expert on the King James Version Bible. "
    "Answer ONLY using the exact text from the provided KJV context. "
    "Do NOT use modern translations, external knowledge, or add information not in the context.\n\n"
    "Context from KJV Bible:\n{context_str}\n\n"
    "Question: {query_str}\n\n"
    "Answer using only the provided KJV context. "
    "Quote the relevant verse(s) exactly, then briefly explain or provide context if it helps understanding."
)

# helps with questions needing multiple verses
query_engine = index.as_query_engine(
    text_qa_template=qa_prompt_tmpl,
    similarity_top_k=6,  # pull a few more verses if needed
)

# After setting query_engine
questions = [
    "Who was Methuselah?",
    "What does John 3:16 say?",
    "What must I do to be saved?",
    "How old was Jesus when he was crucified?",
]

for q in questions:
    print(f"\nQuestion: {q}")
    response = query_engine.query(q)
    print("Answer:", response.response)
    print("Sources:")
    for node in response.source_nodes:
        print(f"  • {node.metadata.get('reference', 'unknown')}")
    print("-" * 60)